## 6.4 多输入多输出通道

### 6.4.1 多输入通道

也就是在每一个输入通道定义一个卷积核，在计算结束后将输出相加，如下图所示。

![](../images/6-4-1.svg)

In [1]:
import torch
from d2l import torch as d2l

def corr2d_multi_in(X, K):
    # 先遍历“X”和“K”的第0个维度（通道维度），再把它们加在一起
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))

In [2]:
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
               [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

### 6.4.2 多输出通道

在每个输出通道首先获取所有输入通道，再以对应该输出通道的卷积核计算的结果，譬如4输出通道即为定义了4个$c_i\times k_h\times k_w$的卷积核张量。

In [5]:
def corr2d_multi_in_out(X, K):
    # 迭代“K”的第0个维度，每次都对输入“X”执行互相关运算。
    # 最后将所有结果都叠加在一起
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

In [6]:
K = torch.stack((K, K + 1, K + 2), 0)
K.shape

torch.Size([3, 2, 2, 2])

In [7]:
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

### 6.4.3 $1\times 1$卷积层

$1\times 1$卷积唯一的计算发生在通道上，可以改变输入输出的通道而不改变大小，类似一个全连接层。同时$1\times 1$卷积层需要的权重维度为$c_o\times c_i$加上一个偏置。如下图所示。

![](../images/6-4-2.svg)

In [8]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    # 全连接层中的矩阵乘法
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))

In [9]:
X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6

### 6.4.4 小结

- 多输入多输出通道可以用来扩展卷积层的模型。
- 当以每像素为基础应用时，$1\times 1$卷积层相当于全连接层。
- $1\times1$卷积层通常用于调整网络层的通道数量和控制模型复杂性。